## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,2021-02-01 13:53:04,-33.9258,18.4232,78.80,36,0,25.32,clear sky
1,1,Hermanus,ZA,2021-02-01 13:55:22,-34.4187,19.2345,73.00,52,0,8.99,clear sky
2,2,Pemba,MZ,2021-02-01 14:01:42,-12.9740,40.5178,86.00,79,20,13.80,few clouds
3,3,Ushuaia,AR,2021-02-01 13:55:00,-54.8000,-68.3000,48.20,76,75,28.77,light rain
4,4,Nanortalik,GL,2021-02-01 13:56:07,60.1432,-45.2371,31.46,92,99,0.54,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,2021-02-01 13:53:04,-33.9258,18.4232,78.80,36,0,25.32,clear sky
2,2,Pemba,MZ,2021-02-01 14:01:42,-12.9740,40.5178,86.00,79,20,13.80,few clouds
5,5,Adrar,MR,2021-02-01 13:56:34,20.5022,-10.0711,84.45,12,97,8.81,overcast clouds
6,6,Rikitea,PF,2021-02-01 13:55:21,-23.1203,-134.9692,78.35,74,1,10.45,clear sky
8,8,Maun,BW,2021-02-01 14:01:44,-19.9833,23.4167,80.60,83,75,5.75,light rain
...,...,...,...,...,...,...,...,...,...,...,...
684,684,Collie,AU,2021-02-01 14:04:14,-33.3667,116.1500,75.00,40,0,1.99,clear sky
685,685,Pontianak,ID,2021-02-01 14:04:14,-0.0333,109.3333,77.00,94,40,3.44,moderate rain
688,688,Sao Joao Da Barra,BR,2021-02-01 13:57:12,-21.6403,-41.0511,87.80,48,0,6.91,clear sky
693,693,Cozumel,MX,2021-02-01 14:04:15,20.5083,-86.9458,75.20,78,20,10.36,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                212
City                   212
Country                210
Date                   212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()


City_ID                210
City                   210
Country                210
Date                   210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,78.80,clear sky,-33.9258,18.4232,
2,Pemba,MZ,86.00,few clouds,-12.9740,40.5178,
5,Adrar,MR,84.45,overcast clouds,20.5022,-10.0711,
6,Rikitea,PF,78.35,clear sky,-23.1203,-134.9692,
8,Maun,BW,80.60,light rain,-19.9833,23.4167,
10,Saint-Philippe,RE,86.00,clear sky,-21.3585,55.7679,
12,Touho,NC,77.23,overcast clouds,-20.7833,165.2333,
15,Puerto Ayora,EC,79.00,broken clouds,-0.7393,-90.3518,
19,Dakar,SN,87.80,clear sky,14.6937,-17.4441,
20,Busselton,AU,78.01,clear sky,-33.6500,115.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,78.80,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
2,Pemba,MZ,86.00,few clouds,-12.9740,40.5178,Guesthouse Samotina
6,Rikitea,PF,78.35,clear sky,-23.1203,-134.9692,Pension Maro'i
8,Maun,BW,80.60,light rain,-19.9833,23.4167,Enviro Villa
10,Saint-Philippe,RE,86.00,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...,...
684,Collie,AU,75.00,clear sky,-33.3667,116.1500,Collie Ridge
685,Pontianak,ID,77.00,moderate rain,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center
688,Sao Joao Da Barra,BR,87.80,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
693,Cozumel,MX,75.20,few clouds,20.5083,-86.9458,Cozumel Palace® All Inclusive Resort


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))